In [1]:
import sys
sys.path.insert(0, "../..")

from MPyDATA_examples.Olesik_et_al_2020.analysis import compute_figure_data
from MPyDATA_examples.Olesik_et_al_2020.physics.equilibrium_drop_growth import PdfEvolver
from MPyDATA_examples.Olesik_et_al_2020.plotter import Plotter
from MPyDATA_examples.Olesik_et_al_2020.coordinates import x_id, x_p2, x_p3,  x_log_of_pn
from MPyDATA_examples.utils.show_plot import show_plot
import numpy as np
import matplotlib

In [2]:
def compute_and_plot(psi_coord, grid_layout, n_bins, GC_max, opt_set, plots, filename):
    results, setup = compute_figure_data(
        psi_coord=psi_coord, 
        grid_layouts=(grid_layout,),
        nr=n_bins,
        GC_max = GC_max,
        opt_set=opt_set
    )
    
    colors = ['red', 'green', 'purple', 'blue', 'orange']
    matplotlib.rcParams.update({'font.size': 16})
    for coord in results.keys():
        out_steps = results[coord]['grid']['out_steps']
        dt = results[coord]['grid']['dt']
        wall_times = results[coord]['wall_time']
        print("elapsed real time: ", wall_times)
        mean_times = [np.nanmean(item) for item in wall_times.values()]
        print("mean time (wrt upwind):", mean_times)
        plotter = Plotter(setup, plots=plots)
        for opt_i, opts in enumerate(results[coord]['numerical'].keys()): 
            plot_data = results[coord]['numerical'][opts]
            m_diff, n_diff = [], []
            for i in range(len(out_steps)):
                mnorm = setup.mixing_ratios[i]
                t = out_steps[i] * dt
                linewidth = 1+1.5*i/len(out_steps)
                if opt_i == 0:
                    plotter.pdf_curve(PdfEvolver(setup.pdf, setup.drdt, t), mnorm)
                    plotter.pdf_histogram(
                        results[coord]['grid']['r'], 
                        results[coord]['analytical'][i], 
                        bin_boundaries = results[coord]['grid']['rh'], 
                        label='analytical', 
                        mnorm=mnorm,
                        color='black',
                        linewidth = linewidth
                    )
                    
                    deltami, deltani = plotter.mass_difference(mnorm, plot_data[i], results[coord]['grid']['rh'])
                    m_diff.append(deltami)
                    n_diff.append(deltani)
                    
                str_repl = [["'n_iters': 1","upwind"],
                            ["'n_iters': 2","MPDATA 2 iterations"],
                            ["'n_iters': 3","MPDATA 3 iterations"],
                            ["'fct': True","+ fct"],
                            ["'iga': True","+ iga"],
                            ["'tot': True","+ tot"],
                            ["{",""],["}",""],[","," "]]                            
                for repl in str_repl:
                    opts = opts.replace(repl[0], repl[1])
                plotter.pdf_histogram(
                    results[coord]['grid']['r'], 
                    plot_data[i], 
                    label=opts, 
                    bin_boundaries=results[coord]['grid']['rh'], 
                    linewidth = linewidth,
                    mnorm=mnorm, color = colors[opt_i]
                )
        print(f"grid_layout: {coord}")
        show_plot(filename)

In [3]:
plot_setup = {"psi_coord":x_p2(), "grid_layout":x_log_of_pn(base=2), "GC_max":.26,"n_bins":75}
plot_setup2 = {"psi_coord":x_id(), "grid_layout":x_id(), "GC_max":.07,"n_bins":75}

In [4]:
compute_and_plot(**plot_setup, opt_set=({"n_iters":1},), plots=('n','m'), filename = 'fig_upwinda.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:   12.2s finished


elapsed real time:  {"{'n_iters': 1}": [nan, 0.02505446623093682, 0.025089605734767026, 0.026611093299134338]}
mean time (wrt upwind): [0.025585055088279397]
Mass difference: 0.19741720488728465 dimensionless %
Number difference: 0.007653969789974127 dimensionless %
Mass difference: 3.338078295687996 dimensionless %
Number difference: 0.00124047169558801 dimensionless %
Mass difference: 5.267960166644459 dimensionless %
Number difference: 0.032798895490924435 dimensionless %
Mass difference: 4.813214671733768 dimensionless %
Number difference: 0.6975093679763827 dimensionless %
grid_layout: x_log_of_pn


In [5]:
compute_and_plot(**plot_setup2, opt_set=({"n_iters":1},), plots=['n','m'],  filename = 'fig_upwindb.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:   10.6s finished


elapsed real time:  {"{'n_iters': 1}": [nan, 0.026262626262626262, 0.024601063829787235, 0.02558001189767995]}
mean time (wrt upwind): [0.025481233996697817]
Mass difference: 0.36793541750237413 dimensionless %
Number difference: 2.8095569687690517e-14 dimensionless %
Mass difference: 0.4738031524986308 dimensionless %
Number difference: 0.006836362361410145 dimensionless %
Mass difference: 0.5923321831645456 dimensionless %
Number difference: 0.027240110707723638 dimensionless %
Mass difference: 0.10668574082613347 dimensionless %
Number difference: 0.26764450429797615 dimensionless %
grid_layout: x_id


In [6]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':3}), plots=['n'], filename = 'fig_mpdatas.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   41.2s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   41.2s finished


elapsed real time:  {"{'n_iters': 1}": [nan, 0.11764705882352941, 0.10465949820788531, 0.10740621994228919], "{'n_iters': 2}": [nan, 0.11764705882352941, 0.10465949820788531, 0.10740621994228919], "{'n_iters': 3}": [nan, 0.11764705882352941, 0.10465949820788531, 0.10740621994228919]}
mean time (wrt upwind): [0.10990425899123464, 0.10990425899123464, 0.10990425899123464]
Mass difference: 0.19741720488728465 dimensionless %
Number difference: 0.007653969789974127 dimensionless %
Mass difference: 3.338078295687996 dimensionless %
Number difference: 0.00124047169558801 dimensionless %
Mass difference: 5.267960166644459 dimensionless %
Number difference: 0.032798895490924435 dimensionless %
Mass difference: 4.813214671733768 dimensionless %
Number difference: 0.6975093679763827 dimensionless %
grid_layout: x_log_of_pn


In [7]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'infinite_gauge':True}), plots=['n'], filename = 'fig_iga.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   43.4s remaining:    0.0s
[Parallel(n_jobs=-2)]: Done   3 out of   3 | elapsed:   43.4s finished


elapsed real time:  {"{'n_iters': 1}": [nan, 0.06318082788671024, 0.06379928315412187, 0.0631612696377044], "{'n_iters': 2}": [nan, 0.06318082788671024, 0.06379928315412187, 0.0631612696377044], "{'n_iters': 2, 'infinite_gauge': True}": [nan, 0.06318082788671024, 0.06379928315412187, 0.0631612696377044]}
mean time (wrt upwind): [0.06338046022617884, 0.06338046022617884, 0.06338046022617884]
Mass difference: 0.19741720488728465 dimensionless %
Number difference: 0.007653969789974127 dimensionless %
Mass difference: 3.338078295687996 dimensionless %
Number difference: 0.00124047169558801 dimensionless %
Mass difference: 5.267960166644459 dimensionless %
Number difference: 0.032798895490924435 dimensionless %
Mass difference: 4.813214671733768 dimensionless %
Number difference: 0.6975093679763827 dimensionless %
grid_layout: x_log_of_pn


In [ ]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'infinite_gauge':True,'flux_corrected_transport':True}), plots=['n'], filename = 'fig_fct.pdf')

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.


In [ ]:
compute_and_plot(**plot_setup, opt_set=({'n_iters': 1},{'n_iters':2},{'n_iters':2,'third_order_terms':True}), plots=['n'], filename = 'fig_tot.pdf')

In [ ]:
compute_and_plot(**plot_setup, opt_set=(
    {'n_iters':1},
    {'n_iters':2},
    {'n_iters':3,'infinite_gauge':True, 'flux_corrected_transport':True,'third_order_terms':True},
), plots=['n'], filename='fig_multiopt.pdf')